In [ ]:
import pandas as pd
import os
from tqdm import tqdm

Getting list of customers

In [ ]:
# customer0 = 4436
# customer1 = 4522
# customer2 = 4704
# customer3 = 6374
# list_of_customers = [4436, 4522, 4704, 6374]
balance_df = pd.read_csv("../RawData/balance_sheet_with_headers.csv")
list_of_customers = list(balance_df['customer_id'].unique())

In [44]:
list_of_customers.sort()

In [ ]:
def get_noncurrent(out_df, x, group, accountBool):
    ser = x[accountBool].groupby(group)['balance'].sum()
    dates = ser.index.values
    
    new_val = []
    for date in out_df['date']:
        if date in dates:
            new_val.append(ser[date])
        else:
            new_val.append(0.0)            
    return new_val 

In [50]:
def get_p1(balance_df, profit_df, raw_balance_df, label):
    try:
        out0 = pd.DataFrame()

        current = balance_df['account_subtype'] == 'Current Assets'
        assets = balance_df['account_type'] == 'Assets'

        out0['date'] = balance_df.groupby(label)['balance'].sum().index
        out0['current_assets'] = balance_df[current].groupby(label)['balance'].sum().values
        out0['non_current_assets'] = get_noncurrent(out0, balance_df, label, assets & ~current)

        liabilities = balance_df['account_type'] == 'Liabilities'
        current_l = balance_df['account_subtype'] == 'Current Liabilities'

        out0['current_liabilities'] = balance_df[current_l].groupby(label)['balance'].sum().values
        out0['non_current_liabilities'] = get_noncurrent(out0, balance_df, label, liabilities & ~current_l)

        equity = balance_df['account_type'] == 'Equity'
        out0['equity'] = balance_df[equity].groupby(label)['balance'].sum().values

        account_subtype = profit_df.loc[profit_df['account_subtype'].str.contains("Revenue", case=False)] 
        account_type = profit_df.loc[profit_df['account_type'].str.contains("Revenue", case=False)]
        revenue = pd.concat([account_subtype, account_type])
        print(revenue.head)
        
        out0['net_revenue'] = get_noncurrent(out0, profit_df, label, revenue)
        
        x = raw_balance_df['account_name'].str.split("|", expand=True)
        out0['accounts_receivable'] = get_noncurrent(out0, raw_balance_df, label, x[2] == "Accounts Receivable")
        boolean_x = ((x[3] == "Cash & cash equivalents") | (x[3] == "Cash & cash equivalents US"))
        out0['cash'] = get_noncurrent(out0, raw_balance_df, label, boolean_x)
        out0['securities'] = get_noncurrent(out0, raw_balance_df, label, x[3] == 'Security Deposits')

        out0['qr_top'] = out0['accounts_receivable'] + out0['cash'] + out0['securities']
        out0.drop(['accounts_receivable', 'cash', 'securities'], axis=1, inplace=True)

        expenses = profit_df['account_type'] == 'Expenses'
        out0['expenses'] = get_noncurrent(out0, profit_df, label, expenses)

        sales = (profit_df['account_subtype'] == 'Sales') | (profit_df['account_subtype'] == 'Sales of Product Income')
        out0['sales'] = get_noncurrent(out0, profit_df, label, sales)
        
        out0['net_income'] = get_noncurrent(out0, balance_df, label, balance_df['account_subtype'] == 'Net Income')
        
        label = 'month' if (label == 'period_date') else label

        path = "../Features/" + str(cid)
        if not os.path.exists(path):
            os.makedirs(path)
        out0.to_csv(f"../Features/" + str(cid) + "/priority1_" + str(label) + ".csv")
    except:
        return 1

In [ ]:
def get_dataframes(cid, raw_balance_df):
    balance_df = pd.read_csv("../OfficialData/" + str(cid) + "/balance.csv")
    profit_df = pd.read_csv("../OfficialData/" + str(cid) + "/profit.csv")
    balance_df[['year', 'month', 'day']] = balance_df['period_date'].str.split('-', expand=True)
    profit_df[['year', 'month', 'day']] = profit_df['period_date'].str.split('-', expand=True)
    # raw_balance_df = pd.read_csv("../RawData/balance_sheet_with_headers.csv")
    # raw_balance_df[['year', 'month', 'day']] = raw_balance_df['period_date'].str.split('-', expand=True)
    raw_balance_df = raw_balance_df[raw_balance_df['customer_id'] == cid]
    return balance_df, profit_df, raw_balance_df

In [ ]:
raw_balance_df = pd.read_csv("../RawData/balance_sheet_with_headers.csv")
raw_balance_df[['year', 'month', 'day']] = raw_balance_df['period_date'].str.split('-', expand=True)

In [51]:
#sp = raw_balance_df.copy(deep=True)
#y = raw_balance_df['account_name'].str.split("|", expand=True)
list_of_bad_companies= []
for c in tqdm(range(10)):
    cid = list_of_customers[c]
    if cid == 7:
        if cid in list_of_customers:
            print("cid " + str(cid))
            try:
                balance_df, profit_df, raw_balance_df_c = get_dataframes(cid, raw_balance_df)
            except:
                list_of_bad_companies.append(cid)
                continue
            a = get_p1(balance_df, profit_df, raw_balance_df_c,'period_date')
            b = get_p1(balance_df, profit_df, raw_balance_df_c, 'year')
            if a or b:
                list_of_bad_companies.append(cid)
            #raw_balance_df = sp.copy(deep=True)

  0%|          | 0/10 [00:00<?, ?it/s]

cid 7
<bound method NDFrame.head of            id  customer_id  account_id period_date    balance  report_id  \
2281  1253321            7       22246  2015-08-01   18648.13       1926   
2282  1253320            7       22247  2015-08-01  243015.37       1926   
2299  3246951            7       22246  2016-04-01    4200.00       1732   
2300  3246950            7       22247  2016-04-01   11000.00       1732   
2423  2921075            7       22247  2016-03-01       0.00       2215   
...       ...          ...         ...         ...        ...        ...   
211    143730            7        5711  2014-05-31   20362.50        108   
212    143371            7        5713  2014-05-31       0.00        108   
213    143682            7        5716  2014-05-31       0.00        108   
214    144466            7        5718  2014-05-31       0.00        108   
215    143624            7        5719  2014-05-31   20018.74        108   

     account_type                       account_sub

100%|██████████| 10/10 [00:00<00:00, 39.01it/s]

<bound method NDFrame.head of            id  customer_id  account_id period_date    balance  report_id  \
2281  1253321            7       22246  2015-08-01   18648.13       1926   
2282  1253320            7       22247  2015-08-01  243015.37       1926   
2299  3246951            7       22246  2016-04-01    4200.00       1732   
2300  3246950            7       22247  2016-04-01   11000.00       1732   
2423  2921075            7       22247  2016-03-01       0.00       2215   
...       ...          ...         ...         ...        ...        ...   
211    143730            7        5711  2014-05-31   20362.50        108   
212    143371            7        5713  2014-05-31       0.00        108   
213    143682            7        5716  2014-05-31       0.00        108   
214    144466            7        5718  2014-05-31       0.00        108   
215    143624            7        5719  2014-05-31   20018.74        108   

     account_type                       account_subtype  

In [ ]:
print(list_of_bad_companies)
print(len(list_of_bad_companies))

In [ ]:
print(len(list_of_customers))

In [ ]:
balance_df[liabilities]['account_subtype'].value_counts()

In [ ]:
balance_df['account_type'].value_counts()

In [ ]:
out0['current_assets'] + out0['non_current_assets'] - out0['current_liabilities'] - out0['non_current_liabilities']

In [ ]:
balance_df[assets]['balance'].sum() - balance_df[liabilities]['balance'].sum()

In [ ]:
balance_df[balance_df['account_type'] == 'Equity']['balance'].sum()

In [ ]:
balance_df[['year', 'month', 'day']] = balance_df['period_date'].str.split('-', expand=True)

In [ ]:
balance_df[assets].groupby('year')['balance'].sum() - balance_df[liabilities].groupby('year')['balance'].sum()

In [ ]:
tt1_df = balance_df[balance_df['year'] == '2021']

In [ ]:
tt1_df['balance_actual'] = tt1_df.balance.diff()
tt1_df['balance_actual'].iloc[0] = 0

In [ ]:
tt1_df

In [ ]:
out0

In [ ]:
assets = tt1_df['account_type'] == 'Assets'
assets_b = tt1_df[assets]['balance_actual'].sum()
assets_b

In [ ]:
liable = balance_df['account_type'] == 'Liabilities'
liable_b = balance_df[liable]['balance_actual'].sum()
liable_b

In [ ]:
equity = balance_df['account_type'] == 'Equity'
equity_b = balance_df[equity]['balance_actual'].sum()
equity_b

In [ ]:
assets_b + liable_b + equity_b